# Data Formating for the Story
In this notebook we are going to focus on the formating of the data that will displayed on the Data Story Website : [tweet-awareness.eu](http://tweet-awareness.eu). As this website will display maps using Jave Script directly, we will have to handle the native data format of the data : `.json`

In [291]:
import pandas as pd
import numpy as np 
import os
import pickle

In [179]:
# We first define our local path to the Git Hub page repository : 
story_path = '../../../Data-Story/'
data_path = '../../../Project Data'
tweet_path = os.path.join(data_path,'Tweets')


## 1. Displaying the twitter results : 
We are going to save the twitter results in `.json` : 
- The twitter raw tweet number
- The twitter Normalized Data

Loading the Summary Data : 

In [618]:
event_list = [ev for ev in os.listdir(tweet_path) if ('.DS_Store' not in ev)]
i = 0
for event in event_list:
    
    i += 1
    summary_path = os.path.join(tweet_path,event,'Geocoded/summary.pickle')
    if i == 1 :
        summary_df = pd.read_pickle(summary_path)
        summary_df.columns = [event]
    else :
        event_df = pd.read_pickle(summary_path)
        event_df.columns = [event]
        summary_df = summary_df.join(event_df)
summary_df.fillna(0.0,inplace=True)

event_country = dict(zip(list(summary_df.columns),["FR", "TR",  "NG","US","BE", "PK", "LB", "NG"]))

# Setting 
all_tweets = summary_df.copy()
summary_norm_df = summary_df.copy()

for event, country in event_country.items():
    all_tweets.loc[country,event] = np.nan
for event in list(summary_df.columns):
    summary_norm_df.loc[:,event]  = summary_df.loc[:,event]/ all_tweets.mean(axis=1, skipna = True)

summary_df['average']=np.round(all_tweets.mean(axis=1, skipna = True).values)
summary_df.head()

,Charlie-Hebdo,Istanbul,Nigeria_2015,Orlando,Bruxelles,Pakistan,Lebanon,Nigeria_2016,average
country,,,,,,,,,
AD,357.0,6.0,0.0,160.0,57.0,1.0,0.0,0.0,73.0
AE,3765.0,528.0,194.0,1804.0,762.0,1534.0,497.0,15.0,1137.0
AF,759.0,27.0,21.0,419.0,120.0,156.0,13.0,14.0,191.0
AG,172.0,2.0,2.0,206.0,22.0,6.0,0.0,0.0,51.0
AI,21.0,0.0,3.0,850.0,12.0,3.0,0.0,1.0,111.0


In [619]:
summary_norm_df.head()

,Charlie-Hebdo,Istanbul,Nigeria_2015,Orlando,Bruxelles,Pakistan,Lebanon,Nigeria_2016
country,,,,,,,,
AD,4.915663,0.082616,0.000000,2.203098,0.784854,0.013769,0.000000,0.000000
AE,3.310254,0.464227,0.170568,1.586108,0.669964,1.348720,0.436971,0.013188
AF,3.971223,0.141269,0.109876,2.192283,0.627861,0.816220,0.068018,0.073250
AG,3.356098,0.039024,0.039024,4.019512,0.429268,0.117073,0.000000,0.000000
AI,0.188764,0.000000,0.026966,7.640449,0.107865,0.026966,0.000000,0.008989


Loading Additional Informations : 

In [620]:
country_data_path = os.path.join(data_path,'country_data.pickle')
c_data_df = pd.read_pickle(country_data_path)
c_data_df.reset_index(inplace=True)
c_data_df.set_index('ISO3',inplace=True)

In [621]:
measurement_df = c_data_df[[('name',''),('ISO2',''),('Internet users',''),('POP',''),('latlng',''),('languages','')]]
measurement_df.columns = measurement_df.columns.droplevel(level=1)
measurement_df = measurement_df.join(summary_df,on='ISO2')
measurement_df.head()


,name,ISO2,Internet users,POP,latlng,languages,Charlie-Hebdo,Istanbul,Nigeria_2015,Orlando,Bruxelles,Pakistan,Lebanon,Nigeria_2016,average
ISO3,,,,,,,,,,,,,,,
ABW,Aruba,AW,99000.0,113648.0,"[12.5, -69.96666666]","[Dutch, Papiamento]",57.0,2.0,1.0,46.0,10.0,0.0,0.0,0.0,14.0
AFG,Afghanistan,AF,2690000.0,33332025.0,"[33, 65]","[Dari, Pashto, Turkmen]",759.0,27.0,21.0,419.0,120.0,156.0,13.0,14.0,191.0
AGO,Angola,AO,2434000.0,20172332.0,"[-12.5, 18.5]",[Portuguese],198.0,5.0,15.0,264.0,45.0,10.0,0.0,1.0,67.0
AIA,Anguilla,AI,12000.0,16752.0,"[18.25, -63.16666666]",[English],21.0,0.0,3.0,850.0,12.0,3.0,0.0,1.0,111.0
ALA,Åland Islands,AX,NaN,NaN,"[60.116667, 19.9]",[Swedish],10.0,0.0,1.0,29.0,7.0,4.0,0.0,0.0,6.0


In [622]:
# Saving the This DF in Json format :
measurement_path = os.path.join(story_path,'I-Measurement','measurement.json')
measurement_json = measurement_df.to_json(force_ascii=False,orient='columns',path_or_buf=measurement_path) #,path_or_buf=measurement_path

In [623]:
measurement_path

'../../../Data-Story/I-Measurement/measurement.json'

In [624]:
measurement_norm_df = []

In [625]:
measurement_norm_df = c_data_df[[('name',''),('ISO2',''),('Internet users',''),('POP',''),('latlng',''),('languages','')]]
measurement_norm_df.columns = measurement_norm_df.columns.droplevel(level=1)
measurement_norm_df = measurement_norm_df.join(summary_norm_df,on='ISO2')
measurement_norm_df.head()

,name,ISO2,Internet users,POP,latlng,languages,Charlie-Hebdo,Istanbul,Nigeria_2015,Orlando,Bruxelles,Pakistan,Lebanon,Nigeria_2016
ISO3,,,,,,,,,,,,,,
ABW,Aruba,AW,99000.0,113648.0,"[12.5, -69.96666666]","[Dutch, Papiamento]",3.931034,0.137931,0.068966,3.172414,0.689655,0.000000,0.000000,0.000000
AFG,Afghanistan,AF,2690000.0,33332025.0,"[33, 65]","[Dari, Pashto, Turkmen]",3.971223,0.141269,0.109876,2.192283,0.627861,0.816220,0.068018,0.073250
AGO,Angola,AO,2434000.0,20172332.0,"[-12.5, 18.5]",[Portuguese],2.944238,0.074349,0.223048,3.925651,0.669145,0.148699,0.000000,0.014870
AIA,Anguilla,AI,12000.0,16752.0,"[18.25, -63.16666666]",[English],0.188764,0.000000,0.026966,7.640449,0.107865,0.026966,0.000000,0.008989
ALA,Åland Islands,AX,NaN,NaN,"[60.116667, 19.9]",[Swedish],1.568627,0.000000,0.156863,4.549020,1.098039,0.627451,0.000000,0.000000


In [626]:
# Saving the This DF in Json format :
measurement_path = os.path.join(story_path,'I-Measurement','measurement_norm.json')
measurement_json = measurement_norm_df.to_json(force_ascii=False,orient='columns',path_or_buf=measurement_path) #,path_or_buf=measurement_path

## 2. The Metrics

In [523]:
metrics_json = {}


### Language

In [524]:
language_df = pd.read_pickle("./../LinkingLanguages/country_dist_languages.pkl")
country_data = c_data_df[[('name',''),('ISO2',''),('Internet users',''),('POP',''),('latlng','')]]
country_data.columns = country_data.columns.droplevel(level=1)
country_codes = country_data[['ISO2']].reset_index().set_index('ISO2')

In [525]:
language_df = language_df.join(country_codes).reset_index().set_index("ISO3").drop("index",axis=1)
old_columns = list(language_df.columns)
new_columns = [country_codes.loc[c_2].values[0] for c_2 in old_columns]
language_df.columns = new_columns

language_json = language_df.to_json(force_ascii=False,orient='columns')
metrics_json['language'] = dict(json.loads(language_json))

language_df.head()

,ABW,AFG,AGO,AIA,ALA,ALB,AND,ARE,ARG,ARM,...,VGB,VIR,VNM,VUT,WLF,WSM,YEM,ZAF,ZMB,ZWE
ISO3,,,,,,,,,,,,,,,,,,,,,
ABW,0.0,17.5,13.5,17.5,16.5,12.5,12.5,inf,13.5,15.0,...,17.5,17.5,inf,18.500000,18.5,17.5,inf,15.5,17.5,16.500000
AFG,17.5,0.0,20.0,15.0,13.0,7.0,16.0,10.000000,18.0,9.5,...,15.0,15.0,inf,17.666667,21.0,15.0,10.000000,14.0,15.0,13.000000
AGO,13.5,20.0,0.0,23.0,21.0,15.0,6.0,inf,6.0,17.5,...,23.0,23.0,inf,20.333333,13.0,23.0,inf,22.0,23.0,21.666667
AIA,17.5,15.0,23.0,0.0,12.0,10.0,19.0,inf,21.0,12.5,...,0.0,0.0,inf,0.000000,24.0,0.0,inf,0.0,0.0,0.000000
ALA,16.5,13.0,21.0,12.0,0.0,8.0,17.0,inf,19.0,10.5,...,12.0,12.0,inf,16.000000,22.0,12.0,inf,11.0,12.0,12.000000


### Real Distance

In [526]:
real_distance_df = pd.read_pickle("./../GeoMetrics/real_distance.pickle")
real_distance_df.columns = real_distance_df.columns.droplevel(level=0)
real_distance_df = real_distance_df.join(country_codes).reset_index().set_index("ISO3").drop("ISO2",axis=1)
real_distance_df = real_distance_df*151
old_columns = list(real_distance_df.columns)
new_columns = [country_codes.loc[c_2].values[0] for c_2 in old_columns]
real_distance_df.columns = new_columns
real_distance_df.head()

real_distance_json = real_distance_df.to_json(force_ascii=False,orient='columns')
metrics_json['real_distance'] = dict(json.loads(real_distance_json))


### Hop Distance

In [527]:
hop_distance_df = pd.read_pickle("./../GeoMetrics/hop_distance.pickle")
hop_distance_df.columns = hop_distance_df.columns.droplevel(level=0)
hop_distance_df = hop_distance_df.join(country_codes).reset_index().set_index("ISO3").drop("ISO2",axis=1)

old_columns = list(hop_distance_df.columns)
new_columns = [country_codes.loc[c_2].values[0] for c_2 in old_columns]
hop_distance_df.columns = new_columns
hop_distance_df.head()

hop_distance_json = hop_distance_df.to_json(force_ascii=False,orient='columns')
metrics_json['hop_distance'] = dict(json.loads(hop_distance_json))

In [528]:
hop_distance_df.head()

,ABW,AFG,AGO,AIA,ALA,ALB,AND,ARE,ARG,ARM,...,VGB,VIR,VNM,VUT,WLF,WSM,YEM,ZAF,ZMB,ZWE
ISO3,,,,,,,,,,,,,,,,,,,,,
ABW,0.000000,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
AFG,inf,0.000000,9.000000,inf,inf,4.000000,6.000000,4.000000,inf,2.000000,...,inf,inf,2.000000,inf,inf,inf,4.000000,11.000000,9.000000,inf
AGO,inf,9.000000,0.000000,inf,inf,9.000000,8.000000,8.000000,inf,8.000000,...,inf,inf,11.000000,inf,inf,inf,8.000000,2.000000,1.000000,inf
AIA,inf,inf,inf,0.000000,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
ALA,inf,inf,inf,inf,0.000000,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf


### Flights Routes Distance 

In [545]:
flight_distance_df = pd.read_pickle("./../GeoMetrics/flight_routes.pickle")
flight_distance_df.columns = flight_distance_df.columns.droplevel(level=0)
flight_distance_df = flight_distance_df.join(country_codes).reset_index().set_index("ISO3").drop("ISO2",axis=1)

#old_columns = list(neib_distance_df.columns)
#new_columns = [country_codes.loc[c_2].values[0] for c_2 in old_columns]
flight_distance_df.columns = new_columns
flight_distance_df.head()

flight_distance_json = flight_distance_df.to_json(force_ascii=False,orient='columns')
metrics_json['flight_distance'] = dict(json.loads(flight_distance_json))

In [546]:
flight_distance_df.head()

,ABW,AFG,AGO,AIA,ALA,ALB,AND,ARE,ARG,ARM,...,VGB,VIR,VNM,VUT,WLF,WSM,YEM,ZAF,ZMB,ZWE
ISO3,,,,,,,,,,,,,,,,,,,,,
ABW,inf,inf,inf,inf,inf,inf,6.81818,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
AFG,inf,inf,inf,inf,inf,inf,inf,24.1379,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
AGO,inf,inf,inf,inf,inf,inf,inf,6.89655,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,10.3448,inf,3.44828
AIA,inf,inf,inf,inf,inf,inf,40,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
ALA,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf


### Neighbor Distance 

In [629]:
neib_distance_df = pd.read_pickle("./../GeoMetrics/neighbor_influence_v2.pickle")
neib_distance_df.columns = neib_distance_df.columns.droplevel(level=0)
neib_distance_df = neib_distance_df.join(country_codes).reset_index().set_index("ISO3").drop("ISO2",axis=1)

old_columns = list(neib_distance_df.columns)
new_columns = [country_codes.loc[c_2].values[0] for c_2 in old_columns]
neib_distance_df.columns = new_columns
neib_distance_df.head()
neib_distance_df.replace(np.nan,np.inf,inplace=True)
neib_distance_df.replace(0.0,np.inf,inplace=True)


neib_distance_json = neib_distance_df.to_json(force_ascii=False,orient='columns')
metrics_json['neib_distance'] = dict(json.loads(neib_distance_json))

In [630]:
neib_distance_df.head()

,ABW,AFG,AGO,AIA,ALA,ALB,AND,ARE,ARG,ARM,...,VGB,VIR,VNM,VUT,WLF,WSM,YEM,ZAF,ZMB,ZWE
ISO3,,,,,,,,,,,,,,,,,,,,,
ABW,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
AFG,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
AGO,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,17.645885,inf
AIA,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
ALA,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf


### Religion

In [581]:
religion_distance_df = pd.read_pickle("./../GeoMetrics/rel_distance_df.pickle")
religion_distance_df = religion_distance_df.join(country_codes).reset_index().set_index("ISO3").drop("index",axis=1)

old_columns = list(religion_distance_df.columns)
new_columns = [country_codes.loc[c_2].values[0] for c_2 in old_columns]
religion_distance_df.columns = new_columns
religion_distance_df.head()

religion_distance_json = religion_distance_df.to_json(force_ascii=False,orient='columns')
metrics_json['religion_distance'] = dict(json.loads(religion_distance_json))

### To JSON in the file

In [632]:
metrics_file = open(os.path.join(story_path,'II-Metrics','metrics.json'), 'w')
json.dump(metrics_json,metrics_file,separators=(',', ':'), indent=4)
metrics_file.close()

In [550]:
metrics_json['neib_distance']

{'ABW': {'ABW': 0.0,
  'AFG': 0.0,
  'AGO': 0.0,
  'AIA': 0.0,
  'ALA': 0.0,
  'ALB': 0.0,
  'AND': 0.0,
  'ARE': 0.0,
  'ARG': 0.0,
  'ARM': 0.0,
  'ASM': 0.0,
  'ATA': 0.0,
  'ATF': 0.0,
  'ATG': 0.0,
  'AUS': 0.0,
  'AUT': 0.0,
  'AZE': 0.0,
  'BDI': 0.0,
  'BEL': 0.0,
  'BEN': 0.0,
  'BFA': 0.0,
  'BGD': 0.0,
  'BGR': 0.0,
  'BHR': 0.0,
  'BHS': 0.0,
  'BIH': 0.0,
  'BLM': 0.0,
  'BLR': 0.0,
  'BLZ': 0.0,
  'BMU': 0.0,
  'BOL': 0.0,
  'BRA': 0.0,
  'BRB': 0.0,
  'BRN': 0.0,
  'BTN': 0.0,
  'BVT': 0.0,
  'BWA': 0.0,
  'CAF': 0.0,
  'CAN': 0.0,
  'CCK': 0.0,
  'CHE': 0.0,
  'CHL': 0.0,
  'CHN': 0.0,
  'CIV': 0.0,
  'CMR': 0.0,
  'COD': 0.0,
  'COG': 0.0,
  'COK': 0.0,
  'COL': 0.0,
  'COM': 0.0,
  'CPV': 0.0,
  'CRI': 0.0,
  'CUB': 0.0,
  'CUW': 0.0,
  'CXR': 0.0,
  'CYM': 0.0,
  'CYP': 0.0,
  'CZE': 0.0,
  'DEU': 0.0,
  'DJI': 0.0,
  'DMA': 0.0,
  'DNK': 0.0,
  'DOM': 0.0,
  'DZA': 0.0,
  'ECU': 0.0,
  'EGY': 0.0,
  'ERI': 0.0,
  'ESH': 0.0,
  'ESP': 0.0,
  'EST': 0.0,
  'ETH': 0.0,

In [576]:
metrics_json.keys()

dict_keys(['language', 'real_distance', 'hop_distance', 'flight_distance', 'neib_distance', 'religion_distance'])

In [456]:
metrics_json['real_distance']

{'ABW': {'ABW': 0.0,
  'AFG': 20613.7129489913,
  'AGO': 13881.615779784,
  'AIA': 1344.6844620579,
  'ALA': 15357.0524427011,
  'ALB': 14250.3133845222,
  'AND': 11703.7025251008,
  'ARE': 18799.3389585935,
  'ARG': 7079.0679600234,
  'ARM': 17849.6974451341,
  'ASM': 15639.0340299059,
  'ATA': 18739.5711502827,
  'ATF': 22985.3744011259,
  'ATG': 1411.6436263431,
  'AUS': 24450.6806673935,
  'AUT': 13633.7624132579,
  'AZE': 18234.4099908771,
  'BDI': 15287.0884944579,
  'BEL': 12579.7702344111,
  'BEN': 10914.1217951021,
  'BFA': 10263.2443715644,
  'BGD': 24217.304698689,
  'BGR': 15061.3835427639,
  'BHR': 18311.8347746379,
  'BHS': 1994.4785781699,
  'BIH': 14108.9154682787,
  'BLM': 1341.2888959877,
  'BLR': 16007.2234638406,
  'BLZ': 2925.5684430816,
  'BMU': 3096.6956965107,
  'BOL': 4517.1916330394,
  'BRA': 4080.4969770721,
  'BRB': 1578.6462440227,
  'BRN': 26507.9060453243,
  'BTN': 24336.0995197599,
  'BVT': 14996.0097817078,
  'BWA': 15115.0807238734,
  'CAF': 13761.0504

In [602]:
c_data_df.columns

MultiIndex(levels=[['GEC_code', 'ISO2', 'ISO3', 'Internet users', 'POP', 'area', 'borders', 'gdp', 'gdp_capita', 'gov_type', 'gov_type_num', 'language_codes', 'languages', 'latlng', 'pop_pov', 'religion', 'unemployment', 'name'], ['', 'animist', 'atheist', 'buddhism', 'catholics', 'hindu', 'jewish', 'muslim', 'oriental', 'ortodox', 'other', 'protestants', 'unaffiliated']],
           labels=[[17, 5, 1, 12, 6, 13, 11, 3, 4, 7, 8, 9, 14, 16, 0, 10, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11, 4, 9, 3, 5, 6, 7, 8, 10, 1, 2, 12]])

In [599]:
metrics_json['religion_distance']['PRT']['ESP']

0.13